In [ ]:
#This librarys is to work with matrices
import pandas as pd 
# This librarys is to work with vectors
import numpy as np
# This library is to create some graphics algorithmn
import seaborn as sns
# to render the graphs
import matplotlib.pyplot as plt
#This library use for building ANN model
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import keras
from tensorflow.keras.utils import to_categorical
from keras.layers import Input
#This library use for data preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# This function makes the plot directly on browser
%matplotlib inline

# Tuning packages

In [ ]:
import optuna
from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll.base import scope
import kerastuner as kt
from sklearn.metrics import log_loss

reference: https://www.kaggle.com/harunshimanto/tps-2021-eda-build-an-artificial-neural-network by Harun-Ur-Rashid

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')
sample_submission = pd.read_csv('../input/tabular-playground-series-may-2021/sample_submission.csv')
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

In [ ]:
for i in range(50):
    mean, std = train[f'feature_{i}'].mean(), train[f'feature_{i}'].std()
    train[f'feature_{i}'] = train[f'feature_{i}'].apply(lambda x : (x-mean)/std)
    test[f'feature_{i}'] = test[f'feature_{i}'].apply(lambda x : (x-mean)/std)
label = {var:index for index, var in enumerate(sorted(train['target'].unique()))}
train['target'] = train['target'].map(label)

target = train['target']
train.drop(['target'], inplace=True, axis=1)
train = train.values
target = target.values
target =  to_categorical(target)
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size = 0.1, random_state = 2, stratify=target)

optimizers list
optimizers['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

# Bayesian Optimization

In [ ]:
def create_model1(hp, num_columns, num_labels):
    model = Sequential()
    model.add(Dense(hp.Int('hidden_units_1', 9, 99, 3), activation='relu',input_dim = num_columns,kernel_initializer='uniform'))
    model.add(Dropout(hp.Float("dropout_rate_1", 0.01, 0.5)))
    model.add(Dense(hp.Int('hidden_units_2', 9, 99, 3), kernel_initializer = 'uniform',activation ='relu'))
    model.add(Dropout(hp.Float("dropout_rate_2", 0.01, 0.5)))
    model.add(Dense(hp.Int('hidden_units_3', 9, 150, 3), kernel_initializer = 'uniform',activation ='relu'))
    model.add(Dropout(hp.Float("dropout_rate_3", 0.01, 0.5)))
    model.add(Dense(num_labels, kernel_initializer='uniform', 
                    activation = hp.Choice('out_activation',['softmax','sigmoid'])))
    model.compile(optimizer = 'sgd', 
                       loss = 'categorical_crossentropy', 
                       metrics = ['accuracy'])
    return model

In [ ]:
params = {'num_columns': 50, 'num_labels': 4}
EPOCHS = 3
MAX_TRIAL = 100
model_fn = lambda hp: create_model1(hp, **params)
tuner = kt.tuners.BayesianOptimization(model_fn, kt.Objective('accuracy', direction='max'), MAX_TRIAL, seed = 2020)
tuner.search(X_train, y_train, epochs=EPOCHS, validation_data = (X_val, y_val))
tuner.results_summary()

In [ ]:
model1 = tuner.get_best_models()[0]
model1.fit(X_train, y_train, epochs = 30, verbose = 0)

**Evaluating the model**

In [ ]:
scores = model1.evaluate(X_train, y_train, batch_size=30)
print("%s: %.2f%%" % (model1.metrics_names[1], scores[1]*100))

In [ ]:
sample_submission[['Class_1','Class_2', 'Class_3', 'Class_4']] = model1.predict(test)
sample_submission.to_csv('BO.csv',index=False)
sample_submission

# Optuna

In [ ]:
def create_model2(num_columns, num_labels, hidden_units_1, hidden_units_2, hidden_units_3,
                dropout_rate_1, dropout_rate_2, dropout_rate_3, out_activation):
    hidden_units = [hidden_units_1, hidden_units_2, hidden_units_3]
    dropout_rates = [dropout_rate_1, dropout_rate_2, dropout_rate_3]
    model = Sequential()
    model.add(Dense(hidden_units[0], activation='relu',input_dim = num_columns, kernel_initializer='uniform'))
    model.add(Dropout(dropout_rates[0]))
    model.add(Dense(hidden_units[1], kernel_initializer = 'uniform',activation ='relu'))
    model.add(Dropout(dropout_rates[1]))
    model.add(Dense(hidden_units[2], kernel_initializer = 'uniform',activation ='relu'))
    model.add(Dropout(dropout_rates[2]))
    model.add(Dense(num_labels, kernel_initializer='uniform', 
                    activation = out_activation))
    model.compile(optimizer = 'sgd', 
                       loss = 'categorical_crossentropy', 
                       metrics = ['accuracy'])
    return model
def objective(trial , X = X_train , y = y_train):   
    params = {'num_columns': 50, 
              'num_labels': 4, 
          'hidden_units_1': trial.suggest_int('hidden_units_1' ,9 , 99), 
          'hidden_units_2': trial.suggest_int('hidden_units_2' ,9 , 99), 
          'hidden_units_3': trial.suggest_int('hidden_units_3' ,9 , 150),
          'dropout_rate_1': trial.suggest_uniform('dropout_rate_1' ,0.01 , 0.5), 
          'dropout_rate_2': trial.suggest_uniform('dropout_rate_2' ,0.01 , 0.5), 
          'dropout_rate_3': trial.suggest_uniform('dropout_rate_3' ,0.01 , 0.5),        
          'out_activation': trial.suggest_categorical('out_activation' , ['softmax','sigmoid'])}
    # pruning_callback = optuna.integration.KerasPruningCallback(trial, monitor = 'accuracy')
    model2 = create_model2(**params)
    model2.fit(X_train, y_train, epochs = 3, verbose = 0
               #, callbacks = [pruning_callback]
              )
    y_pred = model2.predict(X_val)
    ll = log_loss(y_val, y_pred)
    return ll
              
study = optuna.create_study(direction = 'minimize' , study_name = 'Optuna NN'
                        #    , pruner = optuna.pruners.HyperbandPruner()
                           )
study.optimize(objective, n_trials = 100)
print('numbers of the finished trials:' , len(study.trials))
print('the best params:' , study.best_trial.params)
print('the best value:' , study.best_value)

In [ ]:
model2 = create_model2(**study.best_trial.params, num_columns = 50, num_labels = 4)
model2.fit(X_train, y_train, epochs = 30, verbose = 0)

In [ ]:
scores = model2.evaluate(X_train, y_train, batch_size=30)
print("%s: %.2f%%" % (model2.metrics_names[1], scores[1]*100))

In [ ]:
sample_submission[['Class_1','Class_2', 'Class_3', 'Class_4']] = model2.predict(test)
sample_submission.to_csv('Optuna.csv',index=False)
sample_submission

In [ ]:
sample_submission[['Class_1','Class_2', 'Class_3', 'Class_4']] = (model1.predict(test) + model2.predict(test)) / 2
sample_submission.to_csv('avg.csv',index=False)
sample_submission